In [3]:
# import pstats
# p = pstats.Stats('/data/data_geodyn/tmp/ctipe_acceloff_testkamodo_w_ctipe/icesat2_2018336_54hr.ctipe/profileresults.txt')
# p.sort_stats('cumulative').print_stats(100)

In [2]:
# %load_ext autoreload
# %autoreload 2
# import os

# kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py'
# model= 'CTIPe'
# file_dir     = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
# # arg 3
# variable_list = "['rho']"
# z_dependence  = "['ilev']"
# dz            = "['1']"            # 1 request partial derivative of variable 0 is no calculation
# # arg 6
# sat_time      = '1426637500.0' # unix time
# sat_height    = '400.0'        # altitude in km
# sat_lat       = '-25.'         # range of -90 to +90 degrees
# sat_lon       = '90.'          # range of 0 to 360 degrees
# model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
#                                #           of the third syntax choice. This value is used as a maximum allowed time 
#                                #           shift for times requested that are between the model data files.
# high_res      = '0.02'         # precision of plevel interp in meters

# command = ('python'             +' '+
#             kamodo_program_path +' '+
#             model               +' '+          
#             file_dir            +' '+
#             variable_list      
# #             z_dependence        +' '+
# #             dz                  +' '+
# #             sat_time            +' '+
# #             sat_height          +' '+
# #             sat_lat             +' '+
# #             sat_lon             +' '+
# #             model_dt            +' '+
# #             high_res 
#           )
# print(command)
# os.system(command)


### Timeit

In [30]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/')
from SingleSatelliteFlythrough import SingleModelFlythrough

import os

# command = 'python /data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py /data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/ rho ilev 181201 210519 485.4546760656302808    40.18117715905883  -117.07021273480323'
#
### Prepare the command line entry:
####        >>> python 
####            program_path/program_name.py 
####            model data path 
####            file_dir 
####            variable_list  
####            z_dependence 
####            dz 
####            sat_time 
####            sat_height 
####            sat_lat 
####            sat_lon 
####            model_dt 
####            high_res

kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py'
model= 'CTIPe'
file_dir     = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
# arg 3
variable_list = "['rho']"
z_dependence  = "['ilev']"
dz            = "['0']"            # 1 request partial derivative of variable 0 is no calculation
# arg 6
sat_time1      = '181202'#YYMMDD   #'1426637500.0' # unix time
sat_time2      = '  1019'#HHMMSS
# sat_time      = '1543809669.0' # unix time

sat_height    = '300.0'        # altitude in km
sat_lat       = '-25.'         # range of -90 to +90 degrees
sat_lon       = '90.'          # range of 0 to 360 degrees
model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
                               #           of the third syntax choice. This value is used as a maximum allowed time 
                               #           shift for times requested that are between the model data files.
high_res      = '10.0'         # precision of plevel interp in meters

# for i,val in enumerate([sat_time2[0]]):
    
#     if i==0:
#         command = ('python'             +' '+
#                 kamodo_program_path +' '+
#                 model               +' '+
#                 file_dir            +' '+
#                 variable_list)
#     else:
command = ('python'             +' '+
            kamodo_program_path +' '+
            model               +' '+
            file_dir            +' '+
            variable_list       +' '+
            z_dependence        +' '+
            dz                  +' '+
            sat_time1           +' '+
            sat_time2           +' '+
            sat_height          +' '+
            sat_lat             +' '+
            sat_lon             +' '+
            model_dt            +' '+
            high_res 
           )


%timeit os.system(command)

# with open(file_dir+model+'_results.txt', 'r') as f:
#     for line_no, line in enumerate(f):
#         print(line)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.79 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
# from scipy.io import FortranFile

# f = FortranFile('test.unf', 'w')
# f.write_record(np.array(6.025050562515907e-12))
# # f.write_record(np.linspace(0,1,20).reshape((5,4)).T)
# f.close()

# # file = open(file_dir+model+'_results.txt', 'w')
# # for i in range(len(var_list)):
# #     if dz[i]==1:  #if dz was determined for this variable, output both
# #         file.write(f"{var_list[i]}, {results[var_list[i]]:.20f}, {results[var_list[i]+'_dz']:.20f}, {var_units[i]}")
# #     else:  #otherwise, just output the value
# #         file.write(f"{var_list[i]}, {results[var_list[i]]:.20f}, {var_units[i]}")
# #     if i<len(var_list):
# #         file.write('\n' )
# # file.close()


## Use Line_profiler to find and remove the bottlenecks in the Kamodo Code

### CTIPE line-by-lin|e

In [32]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
import os
import sys
import numpy as np


sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/')
import kamodo.pygeodyn_connection.wrapper_utilities as U
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/')

sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_readers/')

import kamodo.pygeodyn_connection.model_wrapper as MW
import ctipe_4D

# import kamodo.pygeodyn_readers.reader_utilities as RU
# import kamodo


sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/')
from SingleSatelliteFlythrough import SingleModelFlythrough

# kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py'
model= 'CTIPe'
file_dir     = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
# arg 3
variable_list = "['rho']"
z_dependence  = "['ilev']"
dz            = "['0']"            # 1 request partial derivative of variable 0 is no calculation
sat_time      = '1543809669.0' # unix time
sat_height    = '300.0'        # altitude in km
sat_lat       = '-25.'         # range of -90 to +90 degrees
sat_lon       = '90.'          # range of 0 to 360 degrees
model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
high_res      = '10.0'         # precision of plevel interp in meters

temp_str = variable_list[1:-1].replace("'","").replace(' ','').replace('"','')
variable_list = temp_str.split(',')   #['rho','N_n']
temp_str = z_dependence[1:-1].replace("'","").replace(' ','').replace('"','')
z_dependence = temp_str.split(',')  #['ilev','ilev']
temp_str = dz[1:-1].replace("'","").replace(' ','').replace('"','')
dz = list(np.array(temp_str.split(','),dtype=int))  #[1,0], later treated as a boolean

sat_time = float(sat_time) #1426637500.0
sat_height = float(sat_height) #400. (in km)
sat_lat  = float(sat_lat) #-25.
sat_lon  = float(sat_lon) #90.
model_dt = float(model_dt)  #450.
high_res = float(high_res)

# print('variable_list',variable_list)
# print('z_dependence',z_dependence)
# print('dz',dz)
# print('sat_time',sat_time)
# print('sat_height',sat_height)
# print('sat_lat',sat_lat)
# print('sat_lon',sat_lon)
# print('model_dt',model_dt)
# print('high_res',high_res)

# %lprun -u 1e-3 -f U.Single_FlyAway SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
%timeit SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)


# %lprun -u 1e-3 -f SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
# %lprun -u 1e-3 -f SingleModelFlythrough SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
# print(result)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
{'rho': 6.03251661900992e-12}
249 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
import cProfile


# if __name__ == '__main__':
#     import cProfile, pstats
#     profiler = cProfile.Profile()
#     profiler.enable()
#     main()
#     profiler.disable()
#     stats = pstats.Stats(profiler).sort_stats('cumtime')
#     stats.print_stats()



cProfile.run("SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)") 


{'rho': 6.03251661900992e-12}
         625070 function calls (601136 primitive calls) in 1.620 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(all)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(any)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argmin)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argsort)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(diff)
        3    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(linspace)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(ndim)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(result_type)
       16    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(searchsorted)
       

        4    0.000    0.000    0.000    0.000 numbers.py:429(ilcm)
      756    0.003    0.000    0.017    0.000 numbers.py:710(_eval_subs)
        3    0.000    0.000    0.000    0.000 numeric.py:1865(isscalar)
       10    0.000    0.000    0.000    0.000 numerictypes.py:285(issubclass_)
        5    0.000    0.000    0.000    0.000 numerictypes.py:359(issubdtype)
        2    0.000    0.000    0.000    0.000 operations.py:406(make_args)
        2    0.000    0.000    0.000    0.000 operations.py:94(_from_args)
        1    0.000    0.000    0.000    0.000 posixpath.py:100(split)
       19    0.000    0.000    0.000    0.000 posixpath.py:41(_get_sep)
        1    0.000    0.000    0.000    0.000 posixpath.py:52(normcase)
       18    0.000    0.000    0.000    0.000 posixpath.py:71(join)
      756    0.001    0.000    0.001    0.000 power.py:341(base)
        1    0.000    0.000    0.000    0.000 power.py:345(exp)
      756    0.010    0.000    0.050    0.000 power.py:758(_eval_subs)

Cprofile which functions that take most time

In [29]:
import cProfile, pstats, io
from pstats import SortKey

pr = cProfile.Profile()
pr.enable()
SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(20)
print(s.getvalue())

{'rho': 6.03251661900992e-12}
         625113 function calls (601179 primitive calls) in 1.627 seconds

   Ordered by: cumulative time
   List reduced from 514 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.628    0.814 /data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3396(run_code)
     10/2    0.000    0.000    1.627    0.814 {built-in method builtins.exec}
        1    0.000    0.000    1.627    1.627 <ipython-input-29-9e6b57067bf4>:6(<module>)
        1    0.000    0.000    1.627    1.627 /data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py:83(SingleModelFlythrough)
        1    0.000    0.000    1.627    1.627 /data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/wrapper_utilities.py:760(Single_FlyAway)
       10    0.043    0.004    1.616    0.162 /data/geodyn_proj/interface_ka

In [28]:
import cProfile, pstats, io
from pstats import SortKey

pr = cProfile.Profile()
pr.enable()
SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_callers(.5, 'kamodo')
ps.print_stats(20)
print(s.getvalue())


{'rho': 6.03251661900992e-12}
   Ordered by: cumulative time
   List reduced from 514 to 257 due to restriction <0.5>
   List reduced from 257 to 36 due to restriction <'kamodo'>

Function                                                                                                                            was called by...
                                                                                                                                        ncalls  tottime  cumtime
/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py:83(SingleModelFlythrough)  <-       1    0.000    1.623  <ipython-input-28-47e241ca7e62>:6(<module>)
/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/wrapper_utilities.py:760(Single_FlyAway)                <-       1    0.000    1.623  /data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py:83(SingleModelFlythrough)
/data/geody

Cprofile which functions Loop times


In [23]:
import cProfile, pstats, io
from pstats import SortKey

pr = cProfile.Profile()
pr.enable()
SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)
pr.disable()
s = io.StringIO()
sortby = SortKey.TIME
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(50)
print(s.getvalue())

{'rho': 6.03251661900992e-12}
         625117 function calls (601183 primitive calls) in 1.625 seconds

   Ordered by: internal time
   List reduced from 514 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     9019    0.195    0.000    0.955    0.000 /data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/core/basic.py:1858(_aresame)
56206/42306    0.120    0.000    0.160    0.000 /data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/core/basic.py:1992(_preorder_traversal)
    89804    0.100    0.000    0.100    0.000 {built-in method builtins.isinstance}
12260/12259    0.082    0.000    0.358    0.000 /data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/core/expr.py:135(__eq__)
    30486    0.066    0.000    0.279    0.000 /data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/core/basic.py:2036(__next__)
    30506    0.065    0.000    0.213    0.000 {built-in method builtins.next}
    27

### TIEGCM line-by-line

In [5]:
# %load_ext autoreload
# %autoreload 2
# %load_ext line_profiler
# import os
# import sys
# import numpy as np


# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/')
# import kamodo.pygeodyn_connection.wrapper_utilities as U
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/')
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_readers/')
# import kamodo.pygeodyn_connection.model_wrapper as MW
# import ctipe_4D
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/')
# from SingleSatelliteFlythrough import SingleModelFlythrough, ModelPrerun



# #--------------------------------------------------------------
# kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/pygeodyn_connection/SingleSatelliteFlythrough.py'
# model= 'TIEGCM'
# file_dir     = '/data/data_geodyn/atmos_models_data/data/data_geodyn/atmos_models_data/tiegcm/2003/'
# # arg 3
# variable_list = "['rho']"
# z_dependence  = "['ilev']"
# dz            = "['0']"            # 1 request partial derivative of variable 0 is no calculation
# #nov16, noon
# # sat_time      = '1068984000.0' # unix time
# sat_time1 = '031116'
# sat_time2 = '120000'
# sat_height    = '300.0'        # altitude in km
# sat_lat       = '-25.'         # range of -90 to +90 degrees
# sat_lon       = '90.'          # range of 0 to 360 degrees
# model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
# high_res      = '10.0'         # precision of plevel interp in meters


# #### UNCOMMENT TO CALL FUNCTION DIRECTLY
# # temp_str = variable_list[1:-1].replace("'","").replace(' ','').replace('"','')
# # variable_list = temp_str.split(',')   #['rho','N_n']
# # temp_str = z_dependence[1:-1].replace("'","").replace(' ','').replace('"','')
# # z_dependence = temp_str.split(',')  #['ilev','ilev']
# # temp_str = dz[1:-1].replace("'","").replace(' ','').replace('"','')
# # dz = list(np.array(temp_str.split(','),dtype=int))  #[1,0], later treated as a boolean
# # sat_time = float(sat_time) #1426637500.0
# # sat_height = float(sat_height) #400. (in km)
# # sat_lat  = float(sat_lat) #-25.
# # sat_lon  = float(sat_lon) #90.
# # model_dt = float(model_dt)  #450.
# # high_res = float(high_res)
# ### CALL OPTIONS
# # %lprun -u 1e-3 -f ModelPrerun ModelPrerun(model, file_dir, variable_list)
# # ModelPrerun(model, file_dir, variable_list)
# # %lprun -u 1e-3 -f U.Single_FlyAway SingleModelFlythrough(model, file_dir, variable_list, z_dependence, dz, sat_time, sat_height, sat_lat, sat_lon, model_dt, high_res)



# command = ('python'             +' '+
#             kamodo_program_path +' '+
#             model               +' '+
#             file_dir            +' '+
#             variable_list    #   +' '+
# #             z_dependence        +' '+
# #             dz                  +' '+
# #             sat_time1           +' '+
# #             sat_time2           +' '+
# #             sat_height          +' '+
# #             sat_lat             +' '+
# #             sat_lon             +' '+
# #             model_dt            +' '+
# #             high_res 
#            )


# os.system(command)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


256